In [3]:
def ler_float_lista(prompt, tamanho_esperado):
    while True:
        try:
            valores = list(map(float, input(prompt).strip().split()))
            if len(valores) != tamanho_esperado:
                raise ValueError(f"Insira {tamanho_esperado} valores.")
            return valores
        except ValueError as e:
            print(f"Erro: {e}. Tente novamente.")

In [4]:
import numpy as np
np.set_printoptions(precision=2, suppress=True)

def imprimir_matriz(matriz):
    print(np.array(matriz))

Z = 80\*x1 + 70\*x2 + 100\*x3 + 16\*x4

x1 +  x2 +  x3 + 4x4 <= 250

0x1 + x2 +  x3 + 2x4 <= 600
      
3x1 + 2x2 + 4x3      <= 500

80 70 100 16

1 1 1 4 <= 250

0 1 1 2 <= 600

3 2 4 0 <= 500

In [5]:
print("RESOLUÇÃO DE PPL COM MÉTODO SIMPLEX (MAXIMIZAÇÃO)")
print("-" * 50)

num_variaveis = int(input("Insira o número de variáveis de decisão: "))
num_restricoes = int(input("Insira o número de restrições: "))

print("\nDigite os coeficientes da função objetivo separados por espaço:")
c = ler_float_lista("Ex: 80 70 100 16\n", num_variaveis)

A = []
b = []
print("\nAgora insira as restrições no formato:")
print("a1 a2 a3 ... an (com coeficientes separados por espaço)")

for i in range(num_restricoes):
    print(f"  Restrição {i + 1}:")
    linha = ler_float_lista("    Coeficientes da restrição: ", num_variaveis)
    bi = float(input("    Termo independente (lado direito): "))
    A.append(linha)
    b.append(bi)

RESOLUÇÃO DE PPL COM MÉTODO SIMPLEX (MAXIMIZAÇÃO)
--------------------------------------------------


Insira o número de variáveis de decisão:  4
Insira o número de restrições:  3



Digite os coeficientes da função objetivo separados por espaço:


Ex: 80 70 100 16
 80 70 100 16



Agora insira as restrições no formato:
a1 a2 a3 ... an (com coeficientes separados por espaço)
  Restrição 1:


    Coeficientes da restrição:  1 1 1 4
    Termo independente (lado direito):  250


  Restrição 2:


    Coeficientes da restrição:  0 1 1 2
    Termo independente (lado direito):  600


  Restrição 3:


    Coeficientes da restrição:  3 2 4 0
    Termo independente (lado direito):  500


In [45]:
def construir_tabela(c):
    tabela_local = [linha + [bi] for linha, bi in zip(restricoes, termos_independentes)]
    linha_objetivo = [-ci for ci in c] + [0] * num_restricoes + [0]
    tabela_local.append(linha_objetivo)
    return tabela_local

In [46]:
funcao_objetivo = []
restricoes = []
termos_independentes = []
num_variaveis = 0
num_restricoes = 0
tabela = []
variaveis_basicas = []

funcao_objetivo = c
num_variaveis = len(c)
num_restricoes = len(b)
termos_independentes = b
restricoes = [linha + [0] * i + [1] + [0] * (len(b) - i - 1) for i, linha in enumerate(A)]
tabela = construir_tabela(c)
variaveis_basicas = [num_variaveis + i for i in range(num_restricoes)]

imprimir_matriz(tabela)

[[   1.    1.    1.    4.    1.    0.    0.  250.]
 [   0.    1.    1.    2.    0.    1.    0.  600.]
 [   3.    2.    4.    0.    0.    0.    1.  500.]
 [ -80.  -70. -100.  -16.    0.    0.    0.    0.]]


In [47]:
def pivoteamento(linha, coluna):
    global tabela
    elemento_pivo = tabela[linha][coluna]
    tabela[linha] = [v / elemento_pivo for v in tabela[linha]]

    for r in range(len(tabela)):
        if r == linha:
            continue
        fator = tabela[r][coluna]
        tabela[r] = [
            tabela[r][i] - fator * tabela[linha][i]
            for i in range(len(tabela[0]))
        ]

In [48]:
def resolver():
    global tabela, variaveis_basicas

    while True:
        ultima_linha = tabela[-1][:-1]

        if all(c >= 0 for c in ultima_linha):
            break

        coluna_pivo = ultima_linha.index(min(ultima_linha))
        razoes = []

        for i in range(num_restricoes):
            if tabela[i][coluna_pivo] > 0:
                razoes.append(tabela[i][-1] / tabela[i][coluna_pivo])
            else:
                razoes.append(float('inf'))

        if all(r == float('inf') for r in razoes):
            raise Exception("Solução ilimitada.")

        linha_pivo = razoes.index(min(razoes))
        variaveis_basicas[linha_pivo] = coluna_pivo
        pivoteamento(linha_pivo, coluna_pivo)

resolver()
imprimir_matriz(tabela)

[[    0.5     1.      0.      8.      2.      0.     -0.5   250. ]
 [   -1.      0.      0.     -2.     -1.      1.      0.    350. ]
 [    0.5     0.      1.     -4.     -1.      0.      0.5     0. ]
 [    5.      0.      0.    144.     40.      0.     15.  17500. ]]


In [49]:
def get_solucao():
    solucao = [0.0] * (num_variaveis + num_restricoes)

    for i, var in enumerate(variaveis_basicas):
        solucao[var] = tabela[i][-1]

    return solucao[:num_variaveis], tabela[-1][-1]

solucao, valor_otimo = get_solucao()
for i, val in enumerate(solucao):
    print(f"x{i + 1} = {val:.2f}")
print(f"Lucro ótimo (Z) = {valor_otimo:.2f}")

x1 = 0.00
x2 = 250.00
x3 = 0.00
x4 = 0.00
Lucro ótimo (Z) = 17500.00


In [50]:
def get_precos_sombra():
    return tabela[-1][num_variaveis:num_variaveis + num_restricoes]
    
for i, ps in enumerate(get_precos_sombra()):
    print(f"Restrição {i + 1}: {ps:.2f}")

Restrição 1: 40.00
Restrição 2: 0.00
Restrição 3: 15.00


In [52]:
def analisar_variacao(delta_b):
    novos_rhs = []
    for i in range(num_restricoes):
        temp = 0
        for j in range(num_restricoes):
            temp += tabela[i][j + len(funcao_objetivo)] * delta_b[j]
        temp += tabela[i][-1]
        novos_rhs.append(temp)

    if all(novos_rhs[i] >= 0 for i in range(len(novos_rhs))):
        precos_sombra = get_precos_sombra()
        delta_z = sum(precos_sombra[i] * delta_b[i] for i in range(num_restricoes))
        novo_z = tabela[-1][-1] + delta_z
        return True, novo_z, precos_sombra, novos_rhs

    return False, 0, [], novos_rhs
    
print("\nAnálise de variação:")
delta_b = ler_float_lista("Insira o Δ para cada restrição: ", num_restricoes)
viavel, novo_z, precos_sombra, resultados = analisar_variacao(delta_b)

if viavel:
    print("\nAlterações viáveis.")
    print(f"Novo lucro ótimo estimado: {novo_z:.2f}")
else:
    print(f"\nA alteração não é viável ({resultados})")


Análise de variação:


Insira o Δ para cada restrição:  -25 -60 -50



Alterações viáveis.
Novo lucro ótimo estimado: 15750.00


In [53]:
print("\nSolução ótima encontrada:")
for i, val in enumerate(solucao):
    print(f"  x{i + 1} = {val:.4f}")
print(f"  Lucro ótimo (Z) = {valor_otimo:.4f}")
print("\nPreços-sombra:")
for i, ps in enumerate(get_precos_sombra()):
    print(f"  Restrição {i + 1}: {ps:.4f}")

print("\nAnálise de variação:")
viavel, novo_z, precos_sombra, resultados = analisar_variacao(delta_b)

if viavel:
    print("  Alterações viáveis.")
    print(f"  Novo lucro ótimo estimado: {novo_z:.2f}")
else:
    print(f"  A alteração não é viável ({resultados})")


Solução ótima encontrada:
  x1 = 0.0000
  x2 = 250.0000
  x3 = 0.0000
  x4 = 0.0000
  Lucro ótimo (Z) = 17500.0000

Preços-sombra:
  Restrição 1: 40.0000
  Restrição 2: 0.0000
  Restrição 3: 15.0000

Análise de variação:
  Alterações viáveis.
  Novo lucro ótimo estimado: 15750.00
